Test implementation of grid search using scikit-learn algorithms before spending time testing it with cuML.

Although scikit-learn has [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) for performing a grid search across various algorithms, it doesn't offer the control I need in terms of trial data output (e.g. what I want is outputting the results of each trial in a denormalized row)

XGBoost note: https://stackoverflow.com/questions/63776921/you-are-running-32-bit-python-on-a-64-bit-osmac-and-xgboost-library-could-not


In [1]:
import os

# import xgboost
import polars as pl
import cuml
import cudf

In [2]:
df = (
    pl.scan_parquet(
        "movie_data_plus_embeds_all.parquet"
    )
    # .select(["tconst", "averageRating", "embedding"])
    .with_columns(averageRating=pl.col("averageRating").cast(pl.Float32))
    .collect()
    .sample(fraction=1.0, shuffle=True, seed=42)
)

df

tconst,startYear,numVotes,averageRating,json,embedding
str,i64,i64,f32,str,"array[f32, 768]"
"""tt0173052""",1999,354,4.1,"""{ ""title"": ""The Prince and t…","[0.046187, 0.006053, … 0.011911]"
"""tt0266288""",1996,1054,7.4,"""{ ""title"": ""Azhakiya Ravanan…","[-0.004875, -0.046969, … 0.017516]"
"""tt6263490""",2020,2713,4.3,"""{ ""title"": ""Getaway"", ""gen…","[0.005363, -0.018672, … 0.015112]"
"""tt10049110""",2019,106,7.8,"""{ ""title"": ""Die Wiese"", ""g…","[-0.009997, -0.029303, … 0.037793]"
"""tt5761612""",2018,133,3.8,"""{ ""title"": ""Woman on the Edg…","[0.020259, -0.031869, … -0.01841]"
…,…,…,…,…,…
"""tt0079376""",1979,168,6.2,"""{ ""title"": ""The Proud Twins""…","[0.062672, -0.009446, … 0.019441]"
"""tt1161064""",2008,1194,3.2,"""{ ""title"": ""Super Capers: Th…","[0.022779, 0.053063, … -0.009691]"
"""tt0179526""",1997,340,5.7,"""{ ""title"": ""Who's the Caboos…","[0.001937, 0.003111, … -0.002453]"


To avoid conversion overhead, store all data as `cuDF` [DataFrames](https://docs.rapids.ai/api/cudf/stable/user_guide/api_docs/api/cudf.dataframe/#cudf.DataFrame).

In [3]:
n_test = 20000

X_train = cudf.DataFrame(df[:-n_test]["embedding"].to_numpy().copy())
X_test = cudf.DataFrame(df[-n_test:]["embedding"].to_numpy().copy())

y_train = cudf.Series(df[:-n_test]["averageRating"].to_numpy().copy())
y_test = cudf.Series(df[-n_test:]["averageRating"].to_numpy().copy())

y_train

0         4.1
1         7.4
2         4.3
3         7.8
4         3.8
         ... 
222547    5.0
222548    6.7
222549    6.4
222550    6.0
222551    6.5
Length: 222552, dtype: float32

Test a simple OLS.


In [4]:
model = cuml.LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mae = cuml.metrics.mean_absolute_error(y_test, y_pred)
mae

0.8323460817337036

## Start Grid Search

- Create a parameter grid
- Run sequentually since this will evenatually be on a single GPU.
- Write the results ([time elapsed](https://stackoverflow.com/a/63232040), train MAE, test MAE) to a dict along with the metadata.
- Load all the data into polars and save as parquet.

Test with a Support Vector Machine since that has tweakable parameters.


In [5]:
import itertools
import time

import numpy as np
from sklearn.svm import SVR
from tqdm.auto import tqdm

Build the grid to get all combinations of parameters. Normally numpy shenanigans would be used here for fast speed but that causes data type issues, e.g. several parameters must be `int`s but numpy shenanigans will force `float`s.


In [6]:
param_grid = {
    "C": np.linspace(start=0.8, stop=1.2, num=5),
    "epsilon": np.linspace(start=0.8, stop=1.2, num=5),
}

In [7]:
[x.tolist() for x in param_grid.values()]

[[0.8, 0.9, 1.0, 1.1, 1.2], [0.8, 0.9, 1.0, 1.1, 1.2]]

In [8]:
param_values = [x.tolist() for x in param_grid.values()]

combos = list(itertools.product(*param_values))
combos

[(0.8, 0.8),
 (0.8, 0.9),
 (0.8, 1.0),
 (0.8, 1.1),
 (0.8, 1.2),
 (0.9, 0.8),
 (0.9, 0.9),
 (0.9, 1.0),
 (0.9, 1.1),
 (0.9, 1.2),
 (1.0, 0.8),
 (1.0, 0.9),
 (1.0, 1.0),
 (1.0, 1.1),
 (1.0, 1.2),
 (1.1, 0.8),
 (1.1, 0.9),
 (1.1, 1.0),
 (1.1, 1.1),
 (1.1, 1.2),
 (1.2, 0.8),
 (1.2, 0.9),
 (1.2, 1.0),
 (1.2, 1.1),
 (1.2, 1.2)]

[zip shenanigans](https://stackoverflow.com/a/33737067) to map back to a list of dicts.


In [9]:
param_dicts = [
    dict(zip(list(param_grid.keys()), combos[i])) for i in range(len(combos))
]
param_dicts

[{'C': 0.8, 'epsilon': 0.8},
 {'C': 0.8, 'epsilon': 0.9},
 {'C': 0.8, 'epsilon': 1.0},
 {'C': 0.8, 'epsilon': 1.1},
 {'C': 0.8, 'epsilon': 1.2},
 {'C': 0.9, 'epsilon': 0.8},
 {'C': 0.9, 'epsilon': 0.9},
 {'C': 0.9, 'epsilon': 1.0},
 {'C': 0.9, 'epsilon': 1.1},
 {'C': 0.9, 'epsilon': 1.2},
 {'C': 1.0, 'epsilon': 0.8},
 {'C': 1.0, 'epsilon': 0.9},
 {'C': 1.0, 'epsilon': 1.0},
 {'C': 1.0, 'epsilon': 1.1},
 {'C': 1.0, 'epsilon': 1.2},
 {'C': 1.1, 'epsilon': 0.8},
 {'C': 1.1, 'epsilon': 0.9},
 {'C': 1.1, 'epsilon': 1.0},
 {'C': 1.1, 'epsilon': 1.1},
 {'C': 1.1, 'epsilon': 1.2},
 {'C': 1.2, 'epsilon': 0.8},
 {'C': 1.2, 'epsilon': 0.9},
 {'C': 1.2, 'epsilon': 1.0},
 {'C': 1.2, 'epsilon': 1.1},
 {'C': 1.2, 'epsilon': 1.2}]

Functionalize.


In [10]:
def build_grid_dict(param_grid):
    param_values = [x.tolist() for x in param_grid.values()]

    combos = list(itertools.product(*param_values))
    param_dicts = [
        dict(zip(list(param_grid.keys()), combos[i])) for i in range(len(combos))
    ]
    return param_dicts


param_grid = {
    "C": np.linspace(start=0.8, stop=1.2, num=5),
    "epsilon": np.linspace(start=0.8, stop=1.2, num=5),
}

build_grid_dict(param_grid)[:5]

[{'C': 0.8, 'epsilon': 0.8},
 {'C': 0.8, 'epsilon': 0.9},
 {'C': 0.8, 'epsilon': 1.0},
 {'C': 0.8, 'epsilon': 1.1},
 {'C': 0.8, 'epsilon': 1.2}]

Do the grid search!


In [11]:
save_folder = "grid_search_results"

os.makedirs(save_folder, exist_ok=True)


# to guard against floating point precision issues when printing params
def round_param_floats(params):
    return {k: round(v, 4) if isinstance(v, float) else v for k, v in params.items()}


def grid_search(model_class, param_grid, model_str):
    param_dicts = build_grid_dict(param_grid)
    result_dicts = []

    for params in tqdm(param_dicts):
        model = model_class(**params)  # fresh model instantiation each run

        fit_start = time.time()
        model.fit(X_train, y_train)
        fit_end = time.time()

        y_pred = model.predict(X_train)
        train_mae = cuml.metrics.mean_absolute_error(y_train, y_pred)

        y_pred = model.predict(X_test)
        test_mae = cuml.metrics.mean_absolute_error(y_test, y_pred)

        result_dicts.append(
            {
                "model": model_str,
                "params": str(round_param_floats(params)),
                "fit_time_ms": (fit_end - fit_start) * 1000,
                "train_mae": train_mae,
                "test_mae": test_mae,
            }
        )
        
    result_df = pl.from_dicts(result_dicts).sort("test_mae")
    result_df.write_parquet(os.path.join(save_folder, model_str) + ".parquet")

    return result_df

In [12]:
param_grid = {
    "C": np.linspace(start=0.8, stop=1.2, num=5),
    "epsilon": np.linspace(start=0.8, stop=1.2, num=5),
}

grid_search(cuml.svm.SVR, param_grid, "SupportVectorRegression")

  0%|          | 0/25 [00:00<?, ?it/s]

model,params,fit_time_ms,train_mae,test_mae
str,str,f64,f64,f64
"""SupportVectorRegression""","""{'C': 1.2, 'epsilon': 0.8}""",15200.226068,0.777683,0.788371
"""SupportVectorRegression""","""{'C': 1.1, 'epsilon': 0.8}""",14388.287783,0.780124,0.789141
"""SupportVectorRegression""","""{'C': 1.0, 'epsilon': 0.8}""",13598.178148,0.782751,0.790053
"""SupportVectorRegression""","""{'C': 0.9, 'epsilon': 0.8}""",12739.237785,0.785584,0.791097
"""SupportVectorRegression""","""{'C': 1.2, 'epsilon': 0.9}""",13377.055407,0.783217,0.79178
…,…,…,…,…
"""SupportVectorRegression""","""{'C': 1.2, 'epsilon': 1.2}""",8685.25672,0.800786,0.802692
"""SupportVectorRegression""","""{'C': 1.1, 'epsilon': 1.2}""",8177.156687,0.802601,0.803367
"""SupportVectorRegression""","""{'C': 1.0, 'epsilon': 1.2}""",7670.574188,0.804539,0.804134


Try a few more linear models.


In [13]:
param_grid = {
    "alpha": np.linspace(start=0.5, stop=2.0, num=20),
}

grid_search(cuml.Ridge, param_grid, "RidgeRegression")

  0%|          | 0/20 [00:00<?, ?it/s]

model,params,fit_time_ms,train_mae,test_mae
str,str,f64,f64,f64
"""RidgeRegression""","""{'alpha': 0.5}""",1397.77565,0.847339,0.833036
"""RidgeRegression""","""{'alpha': 0.5789}""",953.874111,0.8475,0.833183
"""RidgeRegression""","""{'alpha': 0.6579}""",961.264372,0.847659,0.833327
"""RidgeRegression""","""{'alpha': 0.7368}""",942.071199,0.847816,0.833467
"""RidgeRegression""","""{'alpha': 0.8158}""",959.233284,0.847968,0.833604
…,…,…,…,…
"""RidgeRegression""","""{'alpha': 1.6842}""",958.242893,0.849386,0.834862
"""RidgeRegression""","""{'alpha': 1.7632}""",958.913803,0.849495,0.834955
"""RidgeRegression""","""{'alpha': 1.8421}""",961.125374,0.849601,0.835047


In [14]:
param_grid = {
    "alpha": np.array([1e-4, 1e-3, 1e-2]),
    "l1_ratio": np.linspace(start=0.1, stop=0.9, num=3),
    "penalty": np.array(["none", "l1", "l2"]),
}
grid_search(cuml.SGD, param_grid, "SGDRegressor")

  0%|          | 0/27 [00:00<?, ?it/s]

model,params,fit_time_ms,train_mae,test_mae
str,str,f64,f64,f64
"""SGDRegressor""","""{'alpha': 0.0001, 'l1_ratio': …",19727.659941,0.876597,0.86111
"""SGDRegressor""","""{'alpha': 0.0001, 'l1_ratio': …",19141.0532,0.876597,0.86111
"""SGDRegressor""","""{'alpha': 0.0001, 'l1_ratio': …",19625.766516,0.876597,0.86111
"""SGDRegressor""","""{'alpha': 0.001, 'l1_ratio': 0…",19641.644955,0.876597,0.86111
"""SGDRegressor""","""{'alpha': 0.001, 'l1_ratio': 0…",19663.102388,0.876597,0.86111
…,…,…,…,…
"""SGDRegressor""","""{'alpha': 0.01, 'l1_ratio': 0.…",8128.689289,0.947758,0.936085
"""SGDRegressor""","""{'alpha': 0.01, 'l1_ratio': 0.…",8096.364737,0.947758,0.936085
"""SGDRegressor""","""{'alpha': 0.01, 'l1_ratio': 0.…",3828.159809,1.012862,1.00221


In [15]:
param_grid = {
    "n_neighbors": np.arange(start=2, stop=20),

}

grid_search(cuml.neighbors.KNeighborsRegressor, param_grid, "KNNRegressor")

  0%|          | 0/18 [00:00<?, ?it/s]

model,params,fit_time_ms,train_mae,test_mae
str,str,f64,f64,f64
"""KNNRegressor""","""{'n_neighbors': 19}""",96.245527,0.795766,0.827809
"""KNNRegressor""","""{'n_neighbors': 17}""",97.9743,0.791122,0.828301
"""KNNRegressor""","""{'n_neighbors': 16}""",97.131491,0.78856,0.828641
"""KNNRegressor""","""{'n_neighbors': 18}""",96.479654,0.793653,0.828826
"""KNNRegressor""","""{'n_neighbors': 15}""",100.3654,0.785869,0.829106
…,…,…,…,…
"""KNNRegressor""","""{'n_neighbors': 6}""",96.473694,0.729191,0.858683
"""KNNRegressor""","""{'n_neighbors': 5}""",99.721432,0.711877,0.869727
"""KNNRegressor""","""{'n_neighbors': 4}""",96.337318,0.685762,0.884718


Reload all the saved parquets and combine them.

In [19]:
df_comb = pl.read_parquet(os.path.join(save_folder, "*")).sort("test_mae")
df_comb

model,params,fit_time_ms,train_mae,test_mae
str,str,f64,f64,f64
"""SupportVectorRegression""","""{'C': 1.2, 'epsilon': 0.8}""",15200.226068,0.777683,0.788371
"""SupportVectorRegression""","""{'C': 1.1, 'epsilon': 0.8}""",14388.287783,0.780124,0.789141
"""SupportVectorRegression""","""{'C': 1.0, 'epsilon': 0.8}""",13598.178148,0.782751,0.790053
"""SupportVectorRegression""","""{'C': 0.9, 'epsilon': 0.8}""",12739.237785,0.785584,0.791097
"""SupportVectorRegression""","""{'C': 1.2, 'epsilon': 0.9}""",13377.055407,0.783217,0.79178
…,…,…,…,…
"""SGDRegressor""","""{'alpha': 0.01, 'l1_ratio': 0.…",8096.364737,0.947758,0.936085
"""KNNRegressor""","""{'n_neighbors': 2}""",100.136757,0.547434,0.960825
"""SGDRegressor""","""{'alpha': 0.01, 'l1_ratio': 0.…",3828.159809,1.012862,1.00221


In [20]:
df_comb.write_csv("imdb_grid_search.csv")